# Variability of the surface elevation change: Maps

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import download
from matplotlib import colors

plt.style.use("seaborn-v0_8-notebook")

## Define parameters

In [ ]:
domains = ["greenland", "antarctica"]

## Define request

In [ ]:
collection_id = "satellite-ice-sheet-elevation-change"
request = {
    "variable": "all",
    "format": "zip",
    "climate_data_record_type": "icdr",
    "version": "3_0",
}

## Function to cache

In [ ]:
def get_maps(ds):
    (sec_name,) = set(ds.data_vars) & {"sec", "dhdt"}
    da = ds[sec_name]
    da.attrs["long_name"] = "Surface elevation change"

    da_err = ds[f"{sec_name}_uncert"]
    da_err.attrs["long_name"] = "Surface elevation change standard error"

    (mask_name,) = set(ds.data_vars) & {"land_mask", "surface_type"}
    mask = ds[mask_name] > 0
    missing = 100 * (da.sizes["time"] - da.notnull().sum("time")) / da.sizes["time"]
    missing.attrs = {"long_name": "Missing data", "units": "%"}

    year_to_ns = 1.0e9 * 60 * 60 * 24 * 365
    coeffs = []
    da_cumsum = da.cumsum("time") / 12
    for degree, name in enumerate(("linear_trend", "acceleration"), start=1):
        coeff = da_cumsum.polyfit("time", degree)["polyfit_coefficients"].sel(
            degree=degree, drop=True
        )
        coeff = degree * coeff * (year_to_ns**degree)
        coeff.attrs = {
            "units": f"{da.attrs['units'].split('/', 1)[0]} yr$^{{-{degree}}}$",
            "long_name": f"{da.attrs['long_name']} {name}".replace("_", " "),
        }
        coeffs.append(coeff.rename(name))

    ds = xr.merge(
        [
            da.rename("sec"),
            da_err.rename("sec_err"),
            mask.rename("mask"),
            missing.rename("missing"),
            ds["high_slope"],
        ]
        + coeffs
    )
    return ds.mean("time", keep_attrs=True)

## Download and transform data

In [ ]:
datasets = {}
for domain in domains:
    print(f"{domain=}")
    datasets[domain] = download.download_and_transform(
        collection_id,
        request | {"domain": domain},
        transform_func=get_maps,
        invalidate_cache=True,
    ).compute()

## Define plotting functions

In [ ]:
def plot_maps(
    datasets, varname, suptitle=None, vmins=None, vmaxs=None, norms=None, **kwargs
):
    vmins = iter([None] * len(datasets)) if vmins is None else iter(vmins)
    vmaxs = iter([None] * len(datasets)) if vmaxs is None else iter(vmaxs)
    norms = iter([None] * len(datasets)) if norms is None else iter(norms)
    fig, axs = plt.subplots(1, len(datasets), width_ratios=[1, 2], layout="constrained")
    for ax, (domain, ds) in zip(axs, datasets.items()):
        ds[varname].plot.imshow(
            ax=ax, vmin=next(vmins), vmax=next(vmaxs), norm=next(norms), **kwargs
        )
        ds["mask"].plot.contour(ax=ax, colors="k", linewidths=0.5)
        ax.axis("equal")
        ax.axis("off")
        ax.set_title(f"{domain} ice sheet".title())
    if suptitle is not None:
        fig.suptitle(suptitle, y=0, va="top")
    return fig, axs

## Plot maps

In [ ]:
varname_kwargs = {
    "sec": {
        "cmap": "RdBu",
        "robust": True,
        "suptitle": "Multiyear mean surface elevation changes of the ice sheets.",
    },
    "sec_err": {
        "cmap": "Reds",
        "norms": (colors.LogNorm(1.0e-2, 1.0e-1), colors.LogNorm(1.0e-1, 1.0e1)),
        "suptitle": "Multiyear mean surface elevation change standard error of the ice sheets.",
    },
    "missing": {
        "cmap": "Reds",
        "suptitle": "Percentage of missing data of the surface elevation changes of the ice sheets.",
    },
    "linear_trend": {
        "cmap": "RdBu",
        "robust": True,
        "suptitle": "Linear trends (slopes) of the surface elevation changes of the ice sheets.",
    },
    "acceleration": {
        "cmap": "RdBu",
        "robust": True,
        "suptitle": "Quadratic trends (accelerations) of the surface elevation changes of the ice sheets.",
    },
}
for varname, kwargs in varname_kwargs.items():
    fig, axs = plot_maps(datasets, varname=varname, **kwargs)

## Plot boxplot

In [ ]:
fig, axs = plt.subplots(1, 2, layout="constrained")
for ax, (domain, ds) in zip(axs, datasets.items()):
    da = ds.set_coords("high_slope")["sec_err"]
    df = da.to_dataframe()
    df.boxplot(
        by="high_slope",
        ax=ax,
        ylabel=f"{da.attrs['long_name']} [{da.attrs['units']}]",
        xlabel="Slope class",
    )
    if domain.lower() == "antarctica":
        ax.set_ylim([0, 5])
    ax.set_xticklabels(["S<2°", "2°<S<5°", "S>5°"])
    ax.set_title(f"{domain} ice sheet".title())
_ = fig.suptitle(
    "Comparison of the slope class with the surface elevation change standard error."
)